In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from plotly import express as px
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets, metrics, neighbors, preprocessing, tree
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.metrics import (accuracy_score, f1_score, precision_score,make_scorer,
                             r2_score, recall_score, confusion_matrix)
from sklearn.model_selection import cross_validate
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.impute import SimpleImputer
import numpy as np

In [2]:
class Paths:
    data_path=Path(r'../data').resolve().absolute()
    data_raw=data_path.joinpath(r'01_raw')
    path_train=data_raw.joinpath(r'train.csv')
    path_test = data_raw.joinpath(r'test.csv')

In [3]:
df_test = pd.read_csv(Paths.path_test,decimal='.')
df_train = pd.read_csv(Paths.path_train , decimal='.')

In [4]:
df_test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [5]:
df_train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [8]:
X_train_num=df_train.drop(df_train.columns[[0,1,2,3,4,6,12,13]], axis=1)
x_train_cat=df_train.drop(df_train.columns[[0,5,7,8,9,10,11,12,13]], axis=1)
y_train_df=df_train[['Transported']]

In [9]:
 

oe=OrdinalEncoder()

X_transform_cat= oe.fit_transform(x_train_cat)



In [10]:
imp_mean_num = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [11]:
X_transform_num=imp_mean_num.fit_transform(X_train_num)

In [12]:
X_transform_cat=imp_mean_cat.fit_transform(x_train_cat)

In [13]:
X_traintTransColumn_num=pd.DataFrame(X_transform_num,columns=X_train_num.columns)
X_traintTransColumn_cat=pd.DataFrame(X_transform_cat,columns=oe.get_feature_names_out())

In [14]:
X_traint_concat=pd.concat([X_traintTransColumn_num,X_traintTransColumn_cat,y_train_df],axis=1)

In [15]:
X_traint_concat

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet,CryoSleep,Cabin,Destination,VIP,Transported
0,39.0,0.0,0.0,0.0,0.0,0.0,Europa,False,B/0/P,TRAPPIST-1e,False,False
1,24.0,109.0,9.0,25.0,549.0,44.0,Earth,False,F/0/S,TRAPPIST-1e,False,True
2,58.0,43.0,3576.0,0.0,6715.0,49.0,Europa,False,A/0/S,TRAPPIST-1e,True,False
3,33.0,0.0,1283.0,371.0,3329.0,193.0,Europa,False,A/0/S,TRAPPIST-1e,False,False
4,16.0,303.0,70.0,151.0,565.0,2.0,Earth,False,F/1/S,TRAPPIST-1e,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,Europa,False,A/98/P,55 Cancri e,True,False
8689,18.0,0.0,0.0,0.0,0.0,0.0,Earth,True,G/1499/S,PSO J318.5-22,False,False
8690,26.0,0.0,0.0,1872.0,1.0,0.0,Earth,False,G/1500/S,TRAPPIST-1e,False,True
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,Europa,False,E/608/S,55 Cancri e,False,False


In [16]:
X_traint_concat.isnull().sum()

Age             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
VIP             0
Transported     0
dtype: int64

In [17]:
datosSinDuplicados = X_traint_concat.drop_duplicates()

In [18]:
datosSinDuplicados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8676 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           8676 non-null   float64
 1   RoomService   8676 non-null   float64
 2   FoodCourt     8676 non-null   float64
 3   ShoppingMall  8676 non-null   float64
 4   Spa           8676 non-null   float64
 5   VRDeck        8676 non-null   float64
 6   HomePlanet    8676 non-null   object 
 7   CryoSleep     8676 non-null   object 
 8   Cabin         8676 non-null   object 
 9   Destination   8676 non-null   object 
 10  VIP           8676 non-null   object 
 11  Transported   8676 non-null   bool   
dtypes: bool(1), float64(6), object(5)
memory usage: 821.8+ KB


In [19]:
oe=OrdinalEncoder()
os =RandomOverSampler()
X_train_df = datosSinDuplicados.drop( 'Transported', axis=1)
y_train_df = datosSinDuplicados['Transported']


In [20]:

Xov,yov= os.fit_resample(X_train_df, y_train_df)
X_trans = oe.fit_transform(X_train_df)

In [21]:
#dividir el dataset
X_train, X_test, y_train, y_test = train_test_split(X_trans, y_train_df, test_size=0.3, random_state=1)

In [24]:
decision_tree_model = DecisionTreeClassifier(max_depth=7).fit(X_train, y_train)
y_pred = decision_tree_model.predict(X_test)

f1_score1 = f1_score(y_test , y_pred, average='macro')
f1_score1

0.7788921866646458

In [23]:
kFold = model_selection.KFold(n_splits=5)
scoring = 'accuracy'
puntaje= (model_selection.cross_val_score(decision_tree_model, X_train, y_train,  scoring = scoring, cv = kFold))
print (f"( {puntaje.mean()}, {puntaje.std()}")

( 0.7623949668137844, 0.011777538547747601
